In [1]:
import pandas as pd
import vertica_python
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import os

In [2]:
username = os.getenv('VERTICA_USERNAME')
password = os.getenv('VERTICA_PASSWORD')
conn_info = {'host': 'warehouse.analytics.edx.org',
             'port': 5433,
             'user': username,
             'password': password,
             'database': 'warehouse',
             'read_timeout': 600,
             'unicode_error': 'strict',
             'ssl': False}

In [3]:
#DROP TABLE IF EXISTS ahemphill.micromasters_funnel;
#CREATE TABLE ahemphill.micromasters_funnel AS#

#SELECT
#	a.user_id,
#	a.course_id,
#	a.last_enrollment_timestamp,
#	b.program_type,
#	c.path,
#	CAST(c.timestamp AS TIMESTAMP) AS page_view_timestamp,
#	RANK() OVER (PARTITION BY a.user_id, a.course_id ORDER BY CAST(c.timestamp AS TIMESTAMP) DESC) AS page_order
#FROM
#	ahemphill.utm_course_enroll a
#JOIN
#	production.d_program_course b
#ON
#	a.course_id = b.course_id
#	AND b.program_type = 'MicroMasters'
#JOIN
#	experimental_events_run14.event_records c
#ON
#	CAST(a.user_id AS VARCHAR) = CAST(c.user_id AS VARCHAR)
#	AND c.event_type = 'page'
#	AND c.channel = 'client'
#	AND CAST(c.timestamp AS TIMESTAMP) < a.last_enrollment_timestamp
#	AND CAST(c.timestamp AS DATE) = CAST(a.last_enrollment_timestamp AS DATE)#

#DROP TABLE IF EXISTS ahemphill.micromasters_funnel_concat;
#CREATE TABLE ahemphill.micromasters_funnel_concat AS#

#SELECT 
#	user_id,
#	course_id,
#	program_type,
#	CONCAT(CONCAT(CONCAT(CONCAT(CONCAT(CONCAT(CONCAT(CONCAT(first_page, '--->'),second_page),'--->'),third_page),'--->'),fourth_page),'--->'),fifth_page) AS nav_path_length_5,
#	CONCAT(CONCAT(CONCAT(CONCAT(third_page,'--->'),fourth_page),'--->'),fifth_page) AS nav_path_length_3,
#	CONCAT(CONCAT(fourth_page,'--->'),fifth_page) AS nav_path_length_2,
#	fifth_page AS nav_path_length_1
#FROM
#(
#	SELECT
#		user_id,
#		course_id,
#		program_type,
#		MAX(CASE WHEN page_order = 5 THEN path ELSE NULL END) AS first_page,
#		MAX(CASE WHEN page_order = 4 THEN path ELSE NULL END) AS second_page,
#		MAX(CASE WHEN page_order = 3 THEN path ELSE NULL END) AS third_page,
#		MAX(CASE WHEN page_order = 2 THEN path ELSE NULL END) AS fourth_page,
#		MAX(CASE WHEN page_order = 1 THEN path ELSE NULL END) AS fifth_page
#	FROM 
#		ahemphill.micromasters_funnel
#	WHERE 
#		page_order <= 5
#	GROUP BY 1,2,3
#) a


In [4]:
micromasters_funnel_query = """

SELECT
	*
FROM
    ahemphill.micromasters_funnel_concat

"""

In [6]:
connection = vertica_python.connect(**conn_info)
    
cur = connection.cursor('dict')
cur.execute(micromasters_funnel_query)
extract = cur.fetchall()
micromasters_funnel_concat_df = pd.DataFrame(extract)

connection.close()

In [10]:
micromasters_funnel_concat_df.head()

,course_id,nav_path_length_1,nav_path_length_2,nav_path_length_3,nav_path_length_5,program_type,user_id
0,course-v1:LouvainX+Louv2x+1T2017,/course/legal-risk-management-strategies-bux-c...,/course/international-law-louvainx-louv5x-2---...,/course/international-human-rights-law-louvain...,/course/guo-ji-mao-yi-international-economics-...,MicroMasters,1
1,course-v1:LouvainX+Louv5x+1T2017,/course/legal-risk-management-strategies-bux-c...,/course/international-law-louvainx-louv5x-2---...,/course/international-human-rights-law-louvain...,/course/guo-ji-mao-yi-international-economics-...,MicroMasters,1
2,course-v1:ColumbiaX+CSMM.101x+1T2017,/account/finish_auth,/login--->/account/finish_auth,/register--->/login--->/account/finish_auth,/course/robotics-columbiax-csmm-103x--->/micro...,MicroMasters,1000
3,course-v1:ColumbiaX+CSMM.102x+1T2017,/course/machine-learning-columbiax-csmm-102x,/micromasters/columbiax-artificial-intelligenc...,/--->/micromasters/columbiax-artificial-intell...,/account/finish_auth--->/course_modes/choose/c...,MicroMasters,1000
4,course-v1:ColumbiaX+CSMM.103x+1T2017,/micromasters/columbiax-artificial-intelligence,/course_modes/choose/course-v1%3AColumbiaX%2BC...,/dashboard--->/course_modes/choose/course-v1%3...,/micromasters/columbiax-artificial-intelligenc...,MicroMasters,1000


In [11]:
micromasters_funnel_concat_df.to_csv('test.csv')